# SPOR NW 2025: Exploring the Potential of Electronic Medical Records

## Load some made up discharge summaries

In [1]:
import pandas as pd

In [4]:
dsum = pd.read_csv("https://raw.githubusercontent.com/centre-for-health-informatics/SPORNW-2025-EMR-Workshop/refs/heads/master/synthetic_discharge_summaries.csv")
print(dsum.shape)
dsum.head(2)

(20, 4)


,title,summary,hypertension-status,Age
0,Pneumonia (Community-Acquired),The patient is a 67-year-old female with a his...,1,67
1,Acute Cholecystitis,The patient is a 49-year-old male with a BMI o...,0,49


# Data Overview

In [6]:
dsum['hypertension-status'].value_counts()

hypertension-status
0    15
1     5
Name: count, dtype: int64

In [7]:
dsum['Age'].describe()

count    20.000000
mean     45.350000
std      18.204323
min      15.000000
25%      30.500000
50%      47.000000
75%      59.000000
max      72.000000
Name: Age, dtype: float64

In [8]:
dsum['summary'].apply(len).describe()

count      20.000000
mean      721.500000
std       208.127112
min       504.000000
25%       591.000000
50%       665.500000
75%       763.250000
max      1361.000000
Name: summary, dtype: float64

In [10]:
print(dsum['summary'].iloc[0])

The patient is a 24-year-old male with no significant past medical history. He presented with acute onset right lower quadrant abdominal pain, nausea, and fever. He has no known history of gastrointestinal disorders, surgeries, or chronic illness, and is a non-smoker who consumes alcohol occasionally. Family history is non-contributory, with no known family history of abdominal disorders.

However, the patient’s maternal grandmother has a history of hypertension and diabetes, which was noted during family history assessment.

The prognosis is excellent post-appendectomy, as the condition was identified early, and there were no signs of rupture or peritonitis. Barring complications, the patient is expected to return to full function within 2-4 weeks.

The patient underwent a laparoscopic appendectomy within 12 hours of presentation. Intraoperative findings were consistent with uncomplicated appendicitis. Post-operative recovery was uneventful, and he tolerated oral intake on postoperati

# Determining disease status with Natural Language Processing (NLP)

## Keyword Search

In [9]:
htn = dsum['summary'].str.contains('hypertension')
print(htn.sum())
htn.head()

8


0     True
1    False
2    False
3     True
4     True
Name: summary, dtype: bool

## How well does this work?
 - what metrics can we use to assess the performance of our methods

### Accuracy
- What fraction do we get correct, where 1 is perfect and 0 is perfectly wrong

In [ ]:
def acc(reference_labels, our_predictions):
    ll = reference_labels == our_predictions

### PPV

### Sensitivity

### NPV

### Specificity

### Put them all together in one performance function

## Where are we getting it wrong?

### False Positives

### False Negatives

## Regular Expressions
https://www.rexegg.com/regex-quickstart.php

## Negations

## Uncertainty

## Extracting Values

## Family history

## LLM